In [51]:
import os

In [52]:
%pwd

'd:\\Projects\\Machine Learning\\MLOPS'

In [53]:
# os.chdir("../")

In [54]:
%pwd

'd:\\Projects\\Machine Learning\\MLOPS'

In [55]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [56]:
from src.mlProject.constants import *

In [57]:
from src.mlProject.utils.common import read_yaml, create_directories

In [58]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories ([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [59]:
pip install  zipfile

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [60]:
import os 
import urllib.request as request
import zipfile 
from src.mlProject.utils.common import get_size
from src.mlProject import logger

In [61]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
       

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(
                f"File: {filename} downloaded with following info: \n{headers}"
            )
        else:
            logger.info(
                f"File already exists of size: {get_size(Path(self.config.local_data_file))}"
            )
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        create_directories([unzip_path])

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"File extracted at location: {unzip_path}")

In [62]:
try :
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-13 20:57:45,593: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-13 20:57:45,595: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-13 20:57:45,597: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-13 20:57:45,599: INFO: common: Created directory at: artifacts]
[2025-11-13 20:57:45,600: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-11-13 20:57:46,695: INFO: 2601372928: File: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5438:3B3AF:14E3DA:1C6B82:69168C99
Accept-Ranges: bytes
Date: F